In [1]:
import numpy as np
import scipy
import scipy.io as sio
import matplotlib.pyplot as plt
import nibabel as nib

import os
from os.path import join as opj
from os.path import join, exists, split
import time
import urllib.request
import warnings
from tqdm import tqdm
from pprint import pprint
import zipfile
import glob
warnings.filterwarnings('ignore')

import pandas as pd
from nilearn import maskers
from nilearn import plotting
from GOD_decoding_utils import *
from sklearn.model_selection import train_test_split
import bdpy


In [ ]:
base_data_path="/storage/GOD/GOD"
sub = "Subject1"
roi = "ROI_VC"



train_data_path=opj(base_data_path,f"{sub}.h5")

# Initialize kamitani_data_mat with the path to a file containing data for Subject 3
# kamitani_data_mat = f"/data/fMRI/GOD/Subject{sub.split('_')[-1]}.h5"
kamitani_data_mat= f"/storage/GOD/GOD/"

# Initialize imagenet_dir with the path to a directory containing images
imagenet_dir = "/storage/GOD/imagenet/images"

# Initialize test_img_csv and train_img_csv with the paths to two CSV files
test_img_csv = '/storage/GOD/imagenet/images/image_test_id.csv'
train_img_csv = '/storage/GOD/imagenet/images/image_training_id.csv'


# Create a data_handler object using the specified arguments
handler = data_handler(h5_file=train_data_path, test_img_csv=test_img_csv, train_img_csv=train_img_csv)


# Get data using the get_data method of the handler object
train, test, test_avg , train_imaginary,test_imaginary_avg = handler.get_data(normalize=1, roi=roi,imag_data=True)

# Get labels using the get_labels method of the handler object
labels_train, labels_test, labels_imaginary_test = handler.get_labels(imag_data=True)

# Get filenames using the get_filenames method of the handler object
filenames_train, filenames_test = handler.get_filenames()

# Convert filenames_train and filenames_test from arrays to Python lists
filenames_train = [i.item() for i in filenames_train]
filenames_test = [i.item() for i in filenames_test]

# Initialize img_dir_path with the path to a directory containing training images
img_dir_path = "/storage/GOD/imagenet/images/training"

# Initialize image_paths with a list of file paths to the training images
image_paths = [os.path.join(img_dir_path, i) for i in filenames_train]

# Initialize test_image_paths with a list of file paths to the test images
test_image_paths = [os.path.join(imagenet_dir,"test", i) for i in filenames_test]

# # Split the data into train and validation sets using the train_test_split function
# fmri_train, fmri_val, image_train, image_val = train_test_split(Y, image_paths, test_size=0.1, random_state=42)

train.shape, test.shape, test_avg.shape, train_imaginary.shape, test_imaginary_avg.shape, labels_train.shape, labels_test.shape, labels_imaginary_test.shape, len(filenames_train), len(filenames_test), len(image_paths), len(test_image_paths)


((1200, 2049),
 (1750, 2049),
 (50, 2049),
 (500, 2049),
 (50, 2049),
 (1200,),
 (1750,),
 (500,),
 1200,
 1750,
 1200,
 1750)

In [ ]:
unique_images = {}

# Iterate over labels and store the first occurrence of each
for label, path in zip(labels_test, test_image_paths):
    if label not in unique_images:
        unique_images[label] = path  # Store first occurrence only

# Sort by label index (0 → 1 → 2)
sorted_image_paths = [unique_images[i] for i in sorted(unique_images.keys())]

print(sorted_image_paths)

In [19]:
import torch
from torchvision import transforms
from PIL import Image

# Define image transformation
transform = transforms.Compose([
    transforms.ToTensor()  # Convert image to tensor
])

def load_images(image_paths):
    tensors = []
    for path in image_paths:
        image = Image.open(path).convert("RGB")  # Load image and convert to RGB
        tensor = transform(image)  # Apply transformation
        tensors.append(tensor)  # Append to list

    # Stack tensors to maintain order
    return torch.stack(tensors)


train_images = load_images(image_paths)

In [20]:
train_images.shape

torch.Size([1200, 3, 500, 500])

In [ ]:
import torch

train_sub1 = torch.from_numpy(test_avg)

# Save the tensor to a file
torch.save(train_sub1, 'GOD/subjectwise/test_brain_signals_VC_sub1.pt')